<a href="https://colab.research.google.com/github/janbanot/msc-cs-code/blob/main/sem3/DL/DL_2025_Lab4A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Import potrzebnych modułów i funkcji

!uv pip install torchinfo

# Importy z biblioteki standardowej
from collections import defaultdict
from random import random
import math

# Importy z bibliotek zewnętrznych
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn, tensor
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms
from PIL import Image

# Importy do wizualizacji
import matplotlib.pyplot as plt
from matplotlib import cm
from tqdm.notebook import tqdm

# Importy z biblioteki scikit-learn
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, confusion_matrix,
    ConfusionMatrixDisplay, classification_report, roc_auc_score
)
from sklearn.model_selection import train_test_split

# Dodatkowe importy
from torchinfo import summary

In [ ]:
%%html
<!-- Potrzebne dla poprawnego wyświetlania tqdm w VSCode https://stackoverflow.com/a/77566731 -->
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}
</style>

# Sieci konwolucyjne



## Operacja konwolucji

**Konwolucja lub inaczej splot** to działanie określone dla pary funkcji, które daje w wyniku nową funkcję. Splot znajduje liczne zastosowania w przetwarzaniu sygnałów, w tym obrazów. Przykładowo, konwolucja z odpowiednio dobranym filtrem pozwala na rozmywanie lub wyostrzanie obrazu, a także detekcję krawędzi.

W sztucznych sieciach neuronowych przetwarzających dodanie warstwy neuronów realizujących konwolucje pikseli obrazu daje zazwyczaj dobre rezultaty poprawiając zdolność uogólniania,
zmniejszając liczbę parametrów oraz skracając czas treningu.

Poniżej widzimy przykład konwolucji dwóch dyskretnych ciągów
$a = (1, 2, 3)$ oraz $b=(4, 5, 6)$.

In [ ]:
a1, a2, a3 = 1, 2, -1
b1, b2, b3 = 2, 1, 0.5

# Operacja konwolucji polega na przenożeniu (a1, a2, a3) przez
# kolejno przesunięty ciąg (b3, b2, b1)
#
#       a1 a2 a3     =>   a1 * b1
# b3 b2 b1
#
#       a1 a2 a3     =>   a1 * b2 + a2 * b1
#    b3 b2 b1
#
#       a1 a2 a3     =>   a1 * b3 + a2 * b2 + a3 * b3
#       b3 b2 b1
#
#       a1 a2 a3     =>   a2 * b3 + a3 * b2
#          b3 b2 b1
#
#       a1 a2 a3     =>   a2 * b3 + a3 * b2
#          b3 b2 b1
#
#       a1 a2 a3     =>   a3 * b3
#             b3 b2 b1

print(np.convolve([a1, a2, a3], [b1, b2, b3]))

[a1 * b1,
 a1*b2 + a2*b1,
 a1*b3 + a2*b2 + a3*b1,
 a2*b3 + a3*b2,
 a3*b3 ]

Analogiczny efekt uzyskamy korzystając z funkcji `numpy.convolve`

Odpowiednio dobierając wagi jednego z ciągów możemy w wyniku konwolucji otrzymać, np. ciąg będący uśrednieniem sąsiednich wyrazów.

In [ ]:
a = [1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2]
b = [0.5, 0.5]  # (1/2, 1/2) -> uzyskamy efekt uśrednienia sąsiednich elementów a
c = np.convolve(a, b)
c

In [ ]:
def plot_conv1d(inp, out):
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(6, 3))

    axes[0].bar(range(len(inp)+(1 if len(inp) < len(out) else 0)), height=(inp+[0.0] if len(inp) < len(out) else inp))
    axes[0].set_title('Ciąg a')
    axes[1].bar(range(len(out)), height=out)
    axes[1].set_xlabel('Konwolucja a z b');


plot_conv1d(a, c)

### Konwolucje 1d w PyTorch

Analogiczny efekt możelmy uzyskać w `PyTorch` za pomocą klasy `Conv1d`.

**UWAGA**: Konwolucje w `PyTorch` zakładają, że dane przetwarzamy w **grupach** (ang. batch) przykładów, a każdy złożony jest z określonej liczby **kanałów** (ang. channel).

Stąd nasze dane umieszczamy w tensorach z 3 wymiarami, tj.

    x[ indeks przykładu ][ indeks kanału ][ indeks piksela ]

In [ ]:
# @title Przypomnienie -- manipulacja wymiarami w PyTorch
m1 = torch.tensor( [[1, 2, 3],
                    [4, 5, 6]])

print(m1.shape, m1[1][2])

m2 = m1.unsqueeze(0)  # Dodajemy nowy wymiar na pozycji 0
print(m2.shape, m2[0][1][2]) # Dodatkowy indeks na poz. 0

m3 = m2.unsqueeze(0)  # I kolejny
print(m3.shape, m3[0][0][1][2])

m4 = m1.view(1, 1, 2, 3)  # Ten sam efekt co 2 powyższe
print(m4.shape, m4[0][0][1][2])

m4 = m1.view(1, 1, 2, -1)  # Możemy podać -1 dla ostatniego wym.
print(m4.shape, m4[0][0][1][2])

m5 = m4.squeeze()  # Usuń wymiary o rozmiarze 1
print(m5.shape, m5[1][2])

m6 = m1.unsqueeze(2) # Dodajemy nowy wymiar na pozycji 2
print(m6.shape, m6[1][2][0])

m7 = m1.permute(1, 0)  # Zamień kolejność wymiarów wiersze <-> kolumny
print(m7.shape, m7[2][1])


In [ ]:
# Tworzymy 3-wymiarowe tensory z naszych 1-wym. ciągów a i b
at = torch.tensor(a, dtype=torch.float32).view(1, 1, -1)
bt = torch.tensor(b, dtype=torch.float32).view(1, 1, -1)

at.shape, bt.shape, at[0, 0]

Wynik jest "hurtowy", tj. tensor 3d odpowiadający przykładom, kanałom oraz obliczonym wartościom


In [ ]:
out = F.conv1d(at, bt, padding=1)

print(f'{out.shape = }')
print(out[0][0])

In [ ]:
# plot_conv1d(a, out.numpy()[0][0])
plot_conv1d(a, out.squeeze().numpy())

### Wersja obiektowa z `Conv1d`

In [ ]:
# Warstwa Conv1d przyjmuje parametry:
# - in_channels=1 jeden kanał wejściowy
# - out_channels=1 jeden kanał wynikowy
# - kernel_size=2 rozmiar kernela
# Podanie parametru `padding=1` spowoduje dodanie jednej dodatkowej wartości (zero) z każdej strony.
conv = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=2, bias=False, padding=1)

# Ustaw parametry konwolucji
with torch.no_grad():
    conv.weight = nn.Parameter(bt)

c_tensor = conv(at)   # Wykonaj konwolucję

# Odłączenie od grafu obliczeń i konwersja do NumPy
c_pytorch = c_tensor.detach().numpy()[0][0]
# lub
c_pytorch = c_tensor.detach().numpy().squeeze()

print(f'Wynik: {c_pytorch}')

In [ ]:
plot_conv1d(a, c_pytorch)

### Uzupełnianie (padding) i rozmiar kernela

Domyślnie `padding=0`, co powoduje, że wynik jest "krótszy" z każdej strony.

In [ ]:
conv = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=2, bias=False,
                 padding=0)

with torch.no_grad():
    conv.weight = nn.Parameter(bt)

c_tensor = conv(at)   # Wykonaj konwolucję

c_pytorch = c_tensor.detach().squeeze().numpy()
print(f'Wynik: {c_pytorch}')

In [ ]:
plot_conv1d(a, c_pytorch)

Dla kernela o rozmiarze 3 możemy mieć średnią 3 sąsiednich elementów.

In [ ]:
conv = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3, bias=False,
                 padding=1)

with torch.no_grad():
    conv.weight = nn.Parameter(torch.tensor([[[1./3, 1./3, 1./3,]]], dtype=torch.float32))

c_tensor = conv(at)   # Wykonaj konwolucję

# Odłączenie od grafu obliczeń, usunięcie zbędnego wymiaru i konwersja do NumPy
c_pytorch = c_tensor.detach().squeeze().numpy()
print(f'Wynik: {c_pytorch}')

In [ ]:
plot_conv1d(a, c_pytorch)

## Konwolucje 2d

W analogiczny sposób można dokonywać konwolucji dla macierzy,
co jest przydatne przy przetwarzaniu obrazów.
Poniżej widzimy kilka przykładów konwolucji przy pomocy
`torch.nn.Conv2d`

In [ ]:
ex1 = torch.zeros(1, 1, 10, 10)
ex1[0, 0, 3:7, 3:7] = 1  # Kwadrat

ex2 = torch.randn(1, 1, 20, 20)  # Losowo

input = ex2

kernels = [
    torch.tensor([[1, 1, 1],
                  [0, 0, 0],
                  [-1, -1, -1]], dtype=torch.float32).view(1, 1, 3, 3),  # Poziomy

    torch.tensor([[-1, 0, 1],
                  [-1, 0, 1],
                  [-1, 0, 1]], dtype=torch.float32).view(1, 1, 3, 3),  # Pionowy

    torch.tensor([[1, 1, 1],
                  [1, 1, 1],
                  [1, 1, 1]], dtype=torch.float32).view(1, 1, 3, 3) / 9  # Rozmycie (blur)
]

for i, kernel in enumerate(kernels):
    output = F.conv2d(input, kernel, padding=0)
    plt.subplot(1, 4, i+2)
    o = output[0, 0].detach()
    plt.imshow(o, cmap='gray')

print(f'Rozmiar wyjścia: {o.shape}')

plt.subplot(1, 4, 1)
plt.imshow(input[0, 0], cmap='gray')
plt.show()

# Zad. 0

Sprawdź działanie konwolucji dla użytych wcześniej filtrów (kerneli) na obrazie "Lena".

In [ ]:
# Pobierz przykładowy obraz:
!wget --output-document Lenna.png https://upload.wikimedia.org/wikipedia/en/7/7d/Lenna_%28test_image%29.png &> /dev/null

# Podgląd
im = Image.open('Lenna.png')
im

In [ ]:
im = im.convert('L')  # Konwersja na skalę szarości
# Dodatkowo normalizujemy wartości ( / 255), tak aby jasność pikseli była z zakresu [0, 1]
pixels = tensor( np.asarray(im) / 255, dtype=torch.float32 )
pixels.shape, pixels.dtype, pixels[0::100, 0::100]

In [ ]:
# Pomocnicza funkcja do wizualizacji

def nn_output_to_image(out):
    pix = out.detach().numpy().squeeze()  # odłącz (nie chcemy obliczać gradientów) i konwertuj do numpy
    # Normalizuj wartości pikseli do przedziału [0, 255]
    pix = np.interp(pix, (pix.min(), pix.max()), (0, 255))
    return Image.fromarray(np.uint8(pix))

In [ ]:
nn_output_to_image(pixels)

In [ ]:
# @title Rozwiązanie...
# Obraz 'Lena' musi mieć odpowiedni kształt dla F.conv2d:
# [Batch_size, Channels_in, Height, Width]
# Nasz tensor 'pixels' ma kształt [H, W], więc dodajemy dwa wymiary.
input_image = pixels.view(1, 1, pixels.shape[0], pixels.shape[1])

# Tytuły dla wykresów
titles = ["Oryginał (Lena)", "Filtr Poziomy", "Filtr Pionowy", "Filtr Rozmycie"]

# Przygotowanie figury do wizualizacji (1 wiersz, 4 kolumny)
plt.figure(figsize=(20, 6))

# Wyświetlenie obrazu oryginalnego
plt.subplot(1, 4, 1)
plt.imshow(pixels, cmap='gray') # Oryginalny tensor 2D
plt.title(titles[0])
plt.axis('off')

# Przetwarzanie i wyświetlanie wyników dla każdego kernela
for i, kernel in enumerate(kernels):
    # Zastosowanie konwolucji 2D
    # padding=0 oznacza, że obraz wyjściowy będzie nieco mniejszy
    output = F.conv2d(input_image, kernel, padding=0)

    # Pobranie wyniku (tensor [H-2, W-2]) i odłączenie go od grafu obliczeń
    output_tensor = output[0, 0].detach()

    # Wyświetlenie przetworzonego obrazu
    plt.subplot(1, 4, i + 2) # Miejsca 2, 3, 4
    plt.imshow(output_tensor, cmap='gray')
    plt.title(titles[i + 1])
    plt.axis('off')

plt.tight_layout()
plt.show()

# Sprawdzenie rozmiarów
print(f"Oryginalny rozmiar obrazu: {pixels.shape}")
print(f"Rozmiar obrazu po konwolucji 3x3 (padding=0): {output_tensor.shape}")

## Liczba kanałów wyjściowych

Parametr `out_channels` określa ile różnych konwolucji jest obliczanych -- każda
z odrębnymi parametrami (maskami) generuje jeden wektor (macierz) wyjściową.
W ten sposób jeden obraz wejściowy może być przekształcony na wiele sposobów,
**skupiając** się na różnych cechach obrazu.

In [ ]:
torch.manual_seed(1234567)

conv = nn.Conv2d(in_channels=1,  # 1 bo obraz w skali szarości
                 out_channels=2, # 2 kanały wyjściowe
                 kernel_size=3,  # rozmiar "maski"
                 bias=False)     # bez wyrazu wolnego

# Jak widać, mamy dwie odrębne maski (filtry)
list(conv.parameters())

In [ ]:
input = pixels.unsqueeze(0)
out = conv(input)
out.shape

Każdy kanał wyjścia to nowy obraz.

In [ ]:
nn_output_to_image(out[0])

In [ ]:
nn_output_to_image(out[1])

### Obraz kolorowy

In [ ]:
conv = nn.Conv2d(in_channels=3,  # 3 bo RGB
                 out_channels=1, # 1 kanał wyjściowy
                 kernel_size=3,  # rozmiar "maski"
                 bias=False)     # bez wyrazu wolnego

list(conv.parameters())

In [ ]:
im_rgb = Image.open('Lenna.png').resize(size=(256, 256))

# display(im_rgb)

pixels = tensor( np.asarray(im_rgb) / 255, dtype=torch.float32 )
print(f'{pixels.shape = }\nR: {pixels[10, 20, 0] = }\nG: {pixels[10, 20, 1] = }')

# Kolejność wymiarów jest nieodpowiednia, obecnie
# dla każdego koordynatu mamy [x, y, channel] = value
#
# Potrzebujemy [channel, x, y] = value

pixels = pixels.moveaxis(2, 0)
print(f'{pixels.shape = }\nR: {pixels[0, 10, 20] = }\nG: {pixels[1, 10, 20] = }')

out = conv(pixels)  # Obliczamy konwolucję
print(f'{out.shape = }')

In [ ]:
nn_output_to_image(out)

## Próbkowanie obrazu -- MaxPool2d

Oprócz konwolucji, bardzo przydatną operacją jest **próbkowanie w dół**. W przypadku `MaxPool2d` polega ono na wzięciu maksymalnej wartości w oknie wejściowym przesuwanym po obrazie wzdłuż każdego wymiaru. Wielkość okna określa parametr `kernel_size`. Przykładowo, dla `kernel_size=2` otrzymamy na wyjściu obraz dwa razy mniejszy.

In [ ]:
maxpool = nn.MaxPool2d(kernel_size=2)

list(maxpool.parameters())  # MaxPool2d nie ma żadnych parametrów

In [ ]:
input = tensor([
    [1, 0, 2, 0],
    [1, 2, 2, 3],
    [1, 0, 2, 3],
    [1, 4, 2, 5]
], dtype=torch.float32)

maxpool(input[None, :, :])

In [ ]:
im = im.convert('L')  # Konwersja na skalę szarości
# Dodatkowo normalizujemy wartości ( / 255), tak aby jasność pikseli była z zakresu [0, 1]
pixels = tensor( np.asarray(im) / 255, dtype=torch.float32 )

maxpool = nn.MaxPool2d(kernel_size=4)
out = maxpool(pixels.unsqueeze(0).unsqueeze(0))
nn_output_to_image(out)

# Przykład -- klasyfikacja MNIST za pomocą sieci konwolucyjnych

In [ ]:
# Pobieramy zbiór MNIST
# Zbiór treningowy...
train_ds = torchvision.datasets.MNIST(root='./data', train=True, download=True)
# ...i testowy
test_ds = torchvision.datasets.MNIST(root='./data', train=False, download=True)

f'{len(train_ds) = }, {len(test_ds) = }'

In [ ]:
train_ds.data[0]

In [ ]:
for i in range(5):
  img, label = train_ds[i]
  display(img)


## Przygotowanie danych

**Uwaga**, pojedynczy obraz na wejściu sieci będzie miał wymiary
(nr kanału, wysokość, szerokość). W przypadku obrazów w skali szarości
liczba-kanałów jest, oczywiście, równa 1, ale dla obrazów RGB będzie 3, a dla
RGBA 4.

Sieci podajemy albo pojedynczy przykład (obraz) albo zbiór przykładów.
W drugim przypadku, wejście będzie 4-wymiarową tablicą (tensorem).

Zbiór danych ma oryginalnie wymiary (nr obrazu, wys, szer),
więc przekształcamy go na (nr obrazu, nr kanału, wys, szer).

In [ ]:
# Dodatkowy wymiar dla kanału wstawiamy za pomocą
#      |
#      V
# [:, np.newaxis, :, :]

X_train = train_ds.data[:, np.newaxis, :, :] / 255
y_train = train_ds.targets.data

X_test = test_ds.data[:, np.newaxis, :, :] / 255
y_test = test_ds.targets.data

# int [][][][] tab = new int [Liczba obrazow][Liczba kanalow][Wysokosc][Szerokosc]

X_train.shape, y_train.shape, X_test.shape

In [ ]:
X_train, _ , y_train, _ = train_test_split(X_train, y_train, train_size=10000, random_state=42)

In [ ]:
train_loader = DataLoader(list(zip(X_train, y_train)), shuffle=True, batch_size=64)
test_loader = DataLoader(list(zip(X_test, y_test)), shuffle=False, batch_size=256)

# DataLoader pozwala nam iterować po kolejnych partiach w postaci par (wej, wyj).
# Jak widzimy, każda porcja danych składa się ze 64 obrazów wejściowych
# oraz odpowiadających im odpowiedzi.
for inp, ans in train_loader:
  print(inp.shape, ans.shape)
  break

Nasza sieć zawiera teraz warstwy konwolucyjne oraz warstwy w pełni połączone.

In [ ]:
torch.manual_seed(42)  # Reset ziarna generatora, dla powtarzalności wyników

model = nn.Sequential(
  # wej: (1, 28, 28) => wyj: (4, 28, 28):
  # z 1 obrazu 28x28 generujemy 4 obrazy 28x28
  nn.Conv2d(in_channels=1, out_channels=4,
            kernel_size=5,  padding=2),
  nn.ReLU(),

  # wej: (4, 28, 28) => wyj: (4, 14, 14):
  nn.MaxPool2d(kernel_size=2),  # Zmniejszamy rozmiar każdego obrazu (kanału) o połowę

  # Podwajamy liczbę kanałów:
  # wej: (4, 14, 14) => wyj: (8, 14, 14):
  nn.Conv2d(in_channels=4, out_channels=8,
            kernel_size=3,  padding=1),
  nn.ReLU(),

  # wej: (8, 14, 14) => wyj: (8, 7, 7):
  nn.MaxPool2d(kernel_size=2),

  # wej. (8, 7, 7) => wyj (8*7*7, )  <- wektor 1d
  nn.Flatten(),  # Łączymy wszystko w jeden wektor [8 x 7 x 7]

  nn.Linear(8 * 7 * 7, 20),  # 20 neuronów w warstwie ukrytej
  nn.ReLU(),

  nn.Linear(20, 10),  # 10 neuronów w warstwie wyjściowej
)

summary(model)

## Trening sieci

W każdej epoce trenujemy sieć na całym zbiorze treningowym, jednak kolejność
w której pokazywane są obrazy jest za każdym razem losowa.


## GPU or not GPU -- this is (not) a question

Obliczenia będą *domyślnie* wykonywane na procesorze (CPU), ale jeżeli mamy
dostępny procesor graficzny (GPU) wraz z zainstalowanymi bibliotekami, to możemy
zazwyczaj znacznie **przyspieszyć** obliczenia.

W notatnikach `Colab` można wybrać w menu `Środowisko wykonawcze / Zmień typ środowiska wykonawczego` **akcelerator sprzętowy**.

Narzędzie `nvidia-smi` pozwala nam pobrać informacje o dostępnym GPU firmy Nvidia.

In [ ]:
# Obliczenia wykonamy na GPU, jeżeli jest dostępne, a na CPU w przeciwny razie
def get_device():
  return torch.device('cuda' if torch.cuda.is_available() else 'cpu')

get_device()

**GPU** oraz **CPU** mają zazwyczaj odrębną przestrzeń adresową (i fizycznie różne banki pamięci), dlatego konieczne jest **kopiowanie** danych z pamięci głównej do pamięci GPU i z powrotem.

Dzieje się to za pomocą metod:
* `.to(device)` gdzie `device` można uzyskać za pomocą `torch.device()` z parametrem `cuda` lub `cpu`
* `.cpu()` -- kopiowanie do pamięci głównej z pamięci GPU

In [ ]:
def train_model(model, train_loader, test_loader,
                optimizer,
                n_epochs=20, eval_every=1,
                device=None,
                history=None):
    device = device or get_device()

    history = history or defaultdict(list)
    # Przenieś model na określone urządzenie
    model.to(device)

    # Definiowanie funkcji straty
    compute_loss = nn.CrossEntropyLoss()

    # Pętla treningowa
    for epoch in range(n_epochs):
        model.train()  # Ustaw model w tryb treningowy

        total_loss = 0
        total_samples = 0
        total_correct = 0

        # Iteracja po danych treningowych
        for x_batch, y_batch in tqdm(train_loader):
            # Przenieś dane na określone urządzenie
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()  # Wyzerowanie gradientów przed kolejną iteracją
            out = model(x_batch)  # Faza w przód

            loss = compute_loss(out, y_batch)

            loss.backward()   # Faza wsteczna do obliczenia gradientów
            optimizer.step()  # Aktualizacja parametrów modelu

            total_samples += x_batch.shape[0]
            total_loss += loss.item() * x_batch.shape[0]

            predicted = torch.argmax(out, -1)
            total_correct += (predicted == y_batch).sum().item()

        history['train_loss'].append(total_loss / total_samples)
        history['train_accuracy'].append(total_correct / total_samples)

        if epoch % eval_every == 0:  # Ewaluacja na zbiorze testowym
            model.eval()   # Ustaw model w tryb ewaluacji
            total_loss = 0
            total_samples = 0
            total_correct = 0
            with torch.no_grad():  # Wyłączenie obliczania gradientów
                for x_batch, y_batch in test_loader:
                    # Przenieś dane na określone urządzenie
                    x_batch, y_batch = x_batch.to(device), y_batch.to(device)

                    out = model(x_batch)  # Faza w przód

                    loss = compute_loss(out, y_batch)

                    total_samples += x_batch.shape[0]
                    total_loss += loss.item() * x_batch.shape[0]

                    predicted = torch.argmax(out, -1)
                    total_correct += (predicted == y_batch).sum().item()

            history['test_loss'].append(total_loss / total_samples)
            history['test_accuracy'].append(total_correct / total_samples)
            print(f'Epoch: {epoch}\tTrain loss: {history["train_loss"][-1]:.3f}'\
                  f'\tAcc: {history["train_accuracy"][-1]:.3f}'\
                  f'\tTest loss: {history["test_loss"][-1]:.3f}'\
                  f'\tTest acc: {history["test_accuracy"][-1]:.3f}')

    return history


def plot_train_history(history):
    # Wizualizacja historii treningu:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 4))  # 1 wiersz, 2 kolumny

    n_epochs = len(history['train_loss'])
    eval_every = n_epochs // len(history['test_loss'])
    xs = range(0, n_epochs, eval_every)
    # Strata
    ax1.set_ylabel('Strata')
    ax1.set_xlabel('Epoka')
    ax1.plot(history['train_loss'], color='green')
    ax1.plot(xs, history['test_loss'], color='orange')
    ax1.legend(['train', 'test'])

    ax2.set_ylabel('Dokładność')
    ax2.set_xlabel('Epoka')
    ax2.plot(history['train_accuracy'], color='green')
    ax2.plot(xs, history['test_accuracy'], color='orange')
    ax2.legend(['train', 'test'])

    plt.show()

In [ ]:
%%timeit -n1 -r1

torch.manual_seed(42)  # Reset ziarna generatora, dla powtarzalności wyników
history = train_model(model, train_loader, test_loader,
            optimizer=torch.optim.SGD(model.parameters(), lr=0.1),
            n_epochs=10, device=get_device())

plot_train_history(history)

In [ ]:
def show_classification_metrics(model, data_loader, device=None, class_names=None):
    if device is None:
        device = get_device()
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_labels = []

    with torch.no_grad():  # Disable gradient computation
        for x_batch, y_batch in data_loader:
            # Move data to the specified device
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            # Prediction
            out = model(x_batch)
            preds = torch.argmax(out, -1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y_batch.cpu().numpy())

    all_labels = np.array(all_labels)
    all_preds = np.array(all_preds)

    # Determine the unique labels
    if class_names is not None:
        labels = np.arange(len(class_names))
    else:
        labels = np.unique(np.concatenate((all_labels, all_preds)))

    # Compute the confusion matrix
    cm = confusion_matrix(all_labels, all_preds, labels=labels)

    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    num_errors = len(all_labels) - np.sum(all_preds == all_labels)

    # Display the confusion matrix
    if class_names is not None:
        if len(labels) != len(class_names):
            raise ValueError("Number of class names must match number of labels.")
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    else:
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot(cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    # Rotate the x-axis labels by 90 degrees
    plt.setp(disp.ax_.get_xticklabels(), rotation=90)
    plt.show()

    # Display additional information
    print(f'Number of errors: {num_errors}')
    print(f'Accuracy: {accuracy:.3f}')
    print(f'Precision: {precision:.3f}')
    print(f'Recall: {recall:.3f}')
    print(f'F1-score: {f1:.3f}')

In [ ]:
show_classification_metrics(model, test_loader);

In [ ]:
def show_errors( model, data_loader, classes = None, device = None, max_errors_per_class = 10):
    """ Display misclassified examples from a data loader. """

    device = device or get_device()
    model.to(device)
    model.eval()

    errors_per_class = defaultdict(list)

    with torch.no_grad():
        for x_batch, y_batch in data_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            # Forward pass
            outputs = model(x_batch)
            preds = torch.argmax(outputs, dim=1)

            # Identify misclassified indices
            misclassified = preds != y_batch
            misclassified_indices = misclassified.nonzero(as_tuple=False).squeeze()

            # Handle case when there's only one misclassification in the batch
            if misclassified_indices.ndim == 0:
                misclassified_indices = misclassified_indices.unsqueeze(0)

            for idx in misclassified_indices:
                true_label = y_batch[idx].item()
                pred_label = preds[idx].item()

                # Append error if under the max limit for the true class
                if len(errors_per_class[true_label]) < max_errors_per_class:
                    errors_per_class[true_label].append((x_batch[idx].cpu(), pred_label, true_label))

    # Plotting the errors
    for true_class, errors in errors_per_class.items():
        if not errors:
            continue  # Skip if there are no errors for this class

        num_errors = len(errors)
        fig, axs = plt.subplots(1, num_errors, figsize=(1 * num_errors, 1.6))
        fig.suptitle(f'True Class: {classes[true_class] if classes else f"Class {true_class}"}', fontsize=10)

        # Ensure axs is iterable
        if num_errors == 1:
            axs = [axs]

        for ax, (image, pred_label, _) in zip(axs, errors):
            # Handle grayscale and RGB images
            if image.shape[0] == 1:
                ax.imshow(image.squeeze(0), cmap='gray')
            else:
                ax.imshow(image.permute(1, 2, 0))

            pred_class_name = classes[pred_label] if classes else f'{pred_label}'
            ax.set_title(f'Pred: {pred_class_name}', fontsize=8)
            ax.axis('off')

        plt.show()


show_errors(model, test_loader)

## Podgląd warstw konwolucyjnych

In [ ]:
img_idx = 1
img, _ = test_ds[img_idx]
img = img.resize(size=(28*2, 28*2))

display('Obraz wejściowy:')
display(img)

out = X_test[img_idx]

# Iterujemy po kolejnych "modułach" i podglądamy jak wygląda wynik na ich wyjściu
#
# Sequential
# ├─Conv2d: 1-1          0.
# ├─ReLU: 1-2            1.
# ├─MaxPool2d: 1-3       2.
# ├─Conv2d: 1-4          3.
# ├─ReLU: 1-5            4.
# ├─MaxPool2d: 1-6       5. <- kończymy na ostatniej warstwie konwolucyjnej
# ├─Flatten: 1-7         <- spłaszczanie
# ├─Linear: 1-8          <- ukryta warstwa w pełni połączona
# ├─ReLU: 1-9
# ├─Linear: 1-10         <- warstwa wyj.

for i in range(6):  # przejdź po kolejnych "warstwach"
  module_idx = str(i)
  module = model._modules[module_idx]
  out = module(out.to(get_device())).cpu()  # przepuść "out" przez moduł
  num_channels, w, h = out.shape

  fig, axes = plt.subplots(1, num_channels)  # każdy kanał to obraz
  fig.set_figwidth(num_channels * 1.2)

  for i in range(num_channels):
    im = nn_output_to_image(out[i])
    axes[i].imshow(im, cmap='gray')
    axes[i].axis('off')

  print(f'\nPo przejściu przez: {str(module)}:')
  display(fig)
  plt.close()

# Zadanie 1

Wytrenuj prostą **sieć konwolucyjną** dla zadania klasyfikacji obrazów ze zbioru [FashionMNIST](https://github.com/zalandoresearch/fashion-mnist).

Rozważ dwie architektury:
- jak w przykładzie z cyframi
- 2x większa liczba kanałów w warstwach konwolucyjnych

In [ ]:
# Pobieramy zbiór FashionMNIST
# Zbiór treningowy...
fashion_train_ds = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True)
fashion_test_ds = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True)



In [ ]:
f'{len(train_ds) = }, {len(test_ds) = }'

In [ ]:
X_train_f = fashion_train_ds.data.float()[:, np.newaxis, :, :] / 255.0
y_train_f = fashion_train_ds.targets

X_test_f = fashion_test_ds.data.float()[:, np.newaxis, :, :] / 255.0
y_test_f = fashion_test_ds.targets

print(f"Kształt X_train_f przed podziałem: {X_train_f.shape}")
print(f"Kształt X_test_f: {X_test_f.shape}")

X_train_f_sample, _, y_train_f_sample, _ = train_test_split(
    X_train_f, y_train_f, train_size=10000, random_state=42, stratify=y_train_f
)

train_loader_f = DataLoader(list(zip(X_train_f_sample, y_train_f_sample)), shuffle=True, batch_size=64)
test_loader_f = DataLoader(list(zip(X_test_f, y_test_f)), shuffle=False, batch_size=256)

fashion_class_names = [
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'
]

# Sprawdzenie jednej partii danych
for inp, ans in train_loader_f:
  print(f"Kształt partii wejściowej (batch): {inp.shape}")
  print(f"Kształt partii wyjściowej (batch): {ans.shape}")
  break

In [ ]:
## 4 i 8 kanałów

torch.manual_seed(42)
model_1 = nn.Sequential(
  # wej: (1, 28, 28) => wyj: (4, 28, 28)
  nn.Conv2d(in_channels=1, out_channels=4, kernel_size=5, padding=2),
  nn.ReLU(),
  # wej: (4, 28, 28) => wyj: (4, 14, 14):
  nn.MaxPool2d(kernel_size=2),
  # wej: (4, 14, 14) => wyj: (8, 14, 14):
  nn.Conv2d(in_channels=4, out_channels=8, kernel_size=3, padding=1),
  nn.ReLU(),
  # wej: (8, 14, 14) => wyj: (8, 7, 7):
  nn.MaxPool2d(kernel_size=2),
  # wej. (8, 7, 7) => wyj (8*7*7 = 392)
  nn.Flatten(),
  nn.Linear(8 * 7 * 7, 20),
  nn.ReLU(),
  nn.Linear(20, 10),
)

print(summary(model_1, input_size=(64, 1, 28, 28), col_names=["input_size", "output_size", "num_params"], verbose=0))

In [ ]:
history_1 = train_model(
    model_1,
    train_loader_f,
    test_loader_f,
    optimizer=torch.optim.SGD(model_1.parameters(), lr=0.1),
    n_epochs=10,
    device=get_device()
)

In [ ]:
plot_train_history(history_1)
show_classification_metrics(model_1, test_loader_f, device=get_device(), class_names=fashion_class_names)

In [ ]:
## 2x więcej kanałów (8 i 16 kanałów)

torch.manual_seed(42)

model_2 = nn.Sequential(
  # wej: (1, 28, 28) => wyj: (8, 28, 28)
  nn.Conv2d(in_channels=1, out_channels=8, kernel_size=5, padding=2), # 2x
  nn.ReLU(),
  # wej: (8, 28, 28) => wyj: (8, 14, 14):
  nn.MaxPool2d(kernel_size=2),
  # wej: (8, 14, 14) => wyj: (16, 14, 14):
  nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, padding=1), # 2x
  nn.ReLU(),
  # wej: (16, 14, 14) => wyj: (16, 7, 7):
  nn.MaxPool2d(kernel_size=2),
  # wej. (16, 7, 7) => wyj (16*7*7 = 784)
  nn.Flatten(),
  nn.Linear(16 * 7 * 7, 20), # Dopasowany rozmiar wejściowy
  nn.ReLU(),
  nn.Linear(20, 10),
)

print(summary(model_2, input_size=(64, 1, 28, 28), col_names=["input_size", "output_size", "num_params"], verbose=0))

In [ ]:
history_2 = train_model(
    model_2,
    train_loader_f,
    test_loader_f,
    optimizer=torch.optim.SGD(model_2.parameters(), lr=0.1),
    n_epochs=10,
    device=get_device()
)

In [ ]:
plot_train_history(history_2)
show_classification_metrics(model_2, test_loader_f, device=get_device(), class_names=fashion_class_names)